In [1]:
import os
import sys
from datetime import datetime, timedelta
from PySide2.QtCore import QUrl, SIGNAL, QWaitCondition
from PySide2.QtMultimedia import QMediaContent, QMediaPlayer
from PySide2.QtMultimediaWidgets import QVideoWidget
from PySide2.QtWidgets import QApplication, QVBoxLayout, QMainWindow, QWidget, QSizePolicy, QInputDialog
from Worker import Worker
from InputWidget import InputWidget
import requests
import json
from random import randint
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import gspread


class VideoWindow(QMainWindow):
    def __init__(self, parent=None):
        super(VideoWindow, self).__init__(parent)

        self.setWindowTitle("")

        self.worker = Worker()
        self.connect(self.worker, SIGNAL('detect(QString)'), self.rcv)

        self.cur_dur = 0
        self.mediaPlayer = QMediaPlayer(None, QMediaPlayer.VideoSurface)
        # set intro video
        self.mediaPlayer.setMedia(QMediaContent(
            QUrl.fromLocalFile(os.path.abspath("Videos/loop.mp4"))))
        self.mediaPlayer.play()

        videoWidget = QVideoWidget()
        videoWidget.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        wid = QWidget(self)
        self.setCentralWidget(wid)

        layout = QVBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)

        layout.addWidget(videoWidget)
        layout.addWidget(InputWidget())

        wid.setLayout(layout)

        self.mediaPlayer.setVideoOutput(videoWidget)

        self.mediaPlayer.stateChanged.connect(self.mediaStateChanged)
        self.mediaPlayer.durationChanged.connect(self.update_duration)

        self.resize(640, 480)
        # self.showMaximized()

    def play_till_end(self, video_path):
        self.mediaPlayer.setMedia(QMediaContent(
            QUrl.fromLocalFile(os.path.abspath(video_path))))
        self.mediaPlayer.play()
        self.qWait(1000)
        self.qWait(self.cur_dur - 500)

    def qWait(self, t):
        end = datetime.now() + timedelta(milliseconds=t)
        while datetime.now() < end:
            QApplication.processEvents()

    def update_duration(self, duration):
        self.cur_dur = duration

    def try_start_loop(self):
        self.qWait(1000)
        cur_vide_name = os.path.basename(
            self.mediaPlayer.media().canonicalUrl().path())
        playing = self.mediaPlayer.state() == QMediaPlayer.PlayingState
        if not playing and cur_vide_name != 'loop.mp4':
            self.mediaPlayer.setMedia(QMediaContent(
                QUrl.fromLocalFile(os.path.abspath("Videos/loop.mp4"))))
            self.mediaPlayer.play()
            self.qWait(1000)
            self.worker.start()

    def mediaStateChanged(self, state):
        cur_vide_name = os.path.basename(
            self.mediaPlayer.media().canonicalUrl().path())
        playing = state == QMediaPlayer.PlayingState
        if not playing and cur_vide_name == 'loop.mp4':
            self.mediaPlayer.play()

    def rcv(self, msg):
        cur_vide_name = os.path.basename(
            self.mediaPlayer.media().canonicalUrl().path())
        if msg == 'DETECTED' and cur_vide_name == 'loop.mp4':
            self.worker.close()
            res = self.call_api()
            self.process(res)

    def showEvent(self, evt):
        self.worker.start()

    def closeEvent(self, event):
        self.worker.close()

    def call_webhook(self, name, phone):
        otp = randint(1000, 9999)
        print(otp)
        url = f"https://hooks.zapier.com/hooks/catch/754911/b6u2uww?name={name}&mobile={phone}&otp={otp}&premises=1"
        r = requests.post(url)
        if r.status_code == 200:
            print(r.content)
            inp_otp, ok = QInputDialog.getText(
                self, 'Hello', 'Please, Enter confirmation code')
            if ok and otp == inp_otp:
                print('Confirmation succesfull')
            else:
                self.play_till_end("Videos/sorry.mp4")
                inp_otp, ok = QInputDialog.getText(
                    self, 'Hello', 'Please, Enter confirmation code')
                if ok and otp == inp_otp:
                    print('Confirmation succesfull')
                else:
                    self.play_till_end("Videos/sorry.mp4")
                    self.try_start_loop()
                    # inp_otp, ok = QInputDialog.getText(self, 'Hello', 'Please, Enter confirmation code')
                    # if(ok):

                self.try_start_loop()

    def call_api(self):
        user_id = '613b0b563be9fe41e3254a46'
        url = "https://search.facex.io:8443/auth/searchWithRawImage"
        payload = {'user_id': user_id}
        files = [
            ('image', ('file', open('img.jpg', 'rb'), 'application/octet-stream'))
        ]
        headers = {}
        response = requests.request(
            "POST", url, headers=headers, data=payload, files=files)

        response = json.loads(response.text)
        return response

    def process(self, response):
        print(response)
        print('API success: ', response['success'])
        if response['success']:
            # ---------------------------------- Found image in base ---------------------------
            name = response['data']['user'][0]['person_name']
            if os.path.exists(f'Videos/{name}.mp4'):
                # ---------------------------------- Found video by name ---------------------------
                print(f'Here is {name} video')
                if datetime.now().hour > 12:
                    # evening
                    self.play_till_end("Videos/good evening.mp4")
                    self.play_till_end(f"Videos/{name}.mp4")
                else:
                    # morning
                    self.play_till_end("Videos/good morning.mp4")
                    self.play_till_end(f"Videos/{name}.mp4")
                self.try_start_loop()
            else:
                # ---------------------------------- Dont found video by name ---------------------------
                print(f'{name} without his video')
                if datetime.now().hour > 12:
                    # evening
                    self.play_till_end("Videos/good evening.mp4")
                else:
                    # morning
                    self.play_till_end("Videos/good morning.mp4")
                self.try_start_loop()
        else:
            # ---------------------------------- ASK Name and Phone ---------------------------
            print('askname')
            self.play_till_end("Videos/askname.mp4")
            name, ok = QInputDialog.getText(
                self, 'Hello', 'Please, Enter your name')
            if ok:
                print(name)
                self.play_till_end("Videos/thankyou.mp4")
                phone, ok = QInputDialog.getText(
                    self, 'Hello', 'Please, Enter your phone number')
                if ok:

                    print(phone)
                    if os.path.exists(f'Videos/{name}.mp4'):
                        self.play_till_end(f'Videos/thankyou.mp4')
                        self.play_till_end(f'Videos/{name}.mp4')
                    else:
                        webhook_url_creating_video = "https://hooks.zapier.com/hooks/catch/754911/b6u4urm?name={}&mobile=919717659983".format(
                            name)
                        create_vid_request = requests.post(
                            webhook_url_creating_video)

                        self.qWait(30*1000)
                        webhook_url_google_sheets = "https://hooks.zapier.com/hooks/catch/754911/b61db2t?mobile={}".format(
                            phone)
                        r = requests.post(webhook_url_google_sheets)

                        scopes = ['https://spreadsheets.google.com/feeds',
                                  'https://www.googleapis.com/auth/drive']
                        creds = ServiceAccountCredentials.from_json_keyfile_name(
                            'visitor_video_download.json', scopes)
                        client = gspread.authorize(creds)
                        sheet = client.open("Visitorlogtobeused")  # open sheet
                        sheet_instance = sheet.get_worksheet(1)
                        records_data = sheet_instance.get_all_records()
                        df = pd.DataFrame.from_dict(records_data)
                        print(df)
                        tmp_name = ""
                        for i in range(df.shape[0]):
                            if df["Title"][i] == tmp_name:
                                url_vid = df["Download URL"][i]

                        r = requests.get(url_vid, allow_redirects=True)
                        open('DownloadedVideo.mp4', 'wb').write(r.content)

                    self.play_till_end("Videos/verifycode.mp4")
                    # ---------------------------------- Call webhook ---------------------------
                    self.call_webhook(name, phone)
                    self.play_till_end("Videos/thankyou.mp4")
                    self.play_till_end("Videos/registered.mp4")
                    self.try_start_loop()
                else:
                    self.try_start_loop()

            else:
                self.try_start_loop()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    player = VideoWindow()
    player.show()
    sys.exit(app.exec_())


{'status': 'notok', 'code': '1000', 'success': False, 'message': 'No image found.', 'data': {}}
API success:  False
askname
Tyrel
18684786362
  Date            VideoID Title Description Callback ID  \
0       Welcome siddharth                                 
1       Welcome siddharth                                 
2       Welcome siddharth                                 

                                        Download URL Thumbnail Full Name  \
0  https://synthesia-ttv-data.s3-eu-west-1.amazon...                       
1  https://synthesia-ttv-data.s3-eu-west-1.amazon...                       
2  https://synthesia-ttv-data.s3-eu-west-1.amazon...                       

  Face ID  
0          
1          
2          
9615


In [3]:
os.listdir("faces")

['face_0.jpg', 'face_1.jpg', 'face_2.jpg']

In [2]:
print(len(os.listdir("faces")))


0


In [3]:
df

NameError: name 'df' is not defined